# How to Find and Plot Level 2 Data from Multiple Granules on a Map using Python

## Overview

Level 2 data from Low Earth Orbit (LEO) satellites are often organized into \"granules\" that contain spatial sets of data that may not match up with the region of interest.  If the region is smaller than the granule, the user can create a spatial subset.  If the region spans mulitple granules, each granule may be plotted individually, or the data can be aggregated and plotted at once.

This How-To assumes that the user has searched for the granules from the [JPSS-1 CrIS Level 2 CLIMCAPS: Atmosphere cloud and surface geophysical state V2](https://doi.org/10.5067/LESQUBLWS18H) data over the **Gulf of Mexico** on **2021-09-23** within a bounding box defined by the following pairs of longitude and latitude: **(-100, 15)** and **(-75, 30)**.  This data search may be achieved either by using the GES DISC web interface or by performing a granule search as demostrated below using the [Common Metadata Repository (CMR)](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html).

The data plotted in this example is the **Surface Air Temperature** from the CLIMCAPS product, but can be modified to apply to other other Level 2 products.


--- 

*Dataset Citation:*

Chris Barnet (2019), Sounder SIPS: JPSS-1 CrIS Level 2 CLIMCAPS: Atmosphere cloud and surface geophysical state V2, Greenbelt, MD, USA, Goddard Earth Sciences Data and Information Services Center (GES DISC), Accessed: [Data Access Date], 10.5067/LESQUBLWS18H

## Prerequisites

This example code is written in Python 3.9 and requires these libraries and files: 
- numpy
- requests
- netCDF 
- matplotlib
- cartopy  
- netrc file with valid Earthdata Login credentials ([How to Generate Earthdata Prerequisite Files](https://disc.gsfc.nasa.gov/information/howto?keywords=prerequisite&title=How%20to%20Generate%20Earthdata%20Prerequisite%20Files))
- Approval to access the GES DISC archives with your Earthdata credentials (https://disc.gsfc.nasa.gov/earthdata-login)
- wget (https://www.gnu.org/software/wget/)

*Note: If the files to be aggregated are not in netCDF format, a different library (e.g. Xarray) will have to be used.*

### 1. Import Libraries

In [58]:
import glob
import numpy as np
import netCDF4
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import datetime
import os
import pandas as pd
import requests
from dotenv import load_dotenv
from urllib.parse import urlparse
from oco_tools import *

### 2. Extract URLs from the CMR and Download Granules

To search for URLs of the granules of interest, we perform a CMR query using the ShortName (SNDRJ1IML2CCPRET) and Version ID of the dataset as well as the time range and bounding box of the time and region of interest. These URLs will be stored in a list for downloading.

In [51]:
cmr_url = 'https://cmr.earthdata.nasa.gov/search/granules'

# NZ bounding box
latMin = -48
latMax = -34
lonMin = 165
lonMax = 179

short_name = 'OCO2_L2_Lite_FP'

# CMR API query
cmr_response = requests.get(cmr_url, 
                            params={
                                'short_name':short_name,
                                'version': '11r',
                                'temporal': '2021-09-23T00:00:00Z,2021-10-24T00:00:00Z',
                                'bounding_box': f"{lonMin},{latMin},{lonMax},{latMax}",
                                'page_size': 200,
                                },
                            headers={
                                'Accept': 'application/json'
                                }
                            )

# create a list of the data URLs
listofurls = []
for i in range(len(cmr_response.json()['feed']['entry'])):
    link = cmr_response.json()['feed']['entry'][i]['links'][0]['href']
    print(link)
    listofurls.append(link)
# cmr_response.json()

https://data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11r/2021/oco2_LtCO2_210922_B11014Ar_220728230207s.nc4
https://data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11r/2021/oco2_LtCO2_210923_B11014Ar_220728230243s.nc4
https://data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11r/2021/oco2_LtCO2_210924_B11014Ar_220728230312s.nc4
https://data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11r/2021/oco2_LtCO2_210925_B11014Ar_220728230359s.nc4
https://data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11r/2021/oco2_LtCO2_210926_B11014Ar_220728230409s.nc4
https://data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11r/2021/oco2_LtCO2_210927_B11014Ar_220728230457s.nc4
https://data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11r/2021/oco2_LtCO2_210928_B11014Ar_220728230523s.nc4
https://data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11r/2021/oco2_LtCO2_210929_B11014Ar_220728230602s.nc4


The following block of code uses wget to download the Level 2 files in the region of interest to the current directory.

In [56]:
fname='dataurls.txt'
f = open(fname,'w')
for url in listofurls:
    f.write(url+'\n')
f.close()

## Making the URLs file

In [ ]:
import glob
import numpy as np
import netCDF4
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import datetime
import os
import pandas as pd
import requests
from dotenv import load_dotenv
from urllib.parse import urlparse
from oco_tools import *

cmr_url = 'https://cmr.earthdata.nasa.gov/search/granules'

# NZ bounding box
latMin = -48
latMax = -34
lonMin = 165
lonMax = 179

short_name = 'OCO2_L2_Lite_FP'

# CMR API query
cmr_response = requests.get(cmr_url, 
                            params={
                                'short_name':short_name,
                                'version': '11r',
                                'temporal': '2021-09-23T00:00:00Z,2021-10-24T00:00:00Z',
                                'bounding_box': f"{lonMin},{latMin},{lonMax},{latMax}",
                                'page_size': 200,
                                },
                            headers={
                                'Accept': 'application/json'
                                }
                            )

# create a list of the data URLs
listofurls = []
for i in range(len(cmr_response.json()['feed']['entry'])):
    link = cmr_response.json()['feed']['entry'][i]['links'][0]['href']
    print(link)
    listofurls.append(link)
# cmr_response.json()

fname='dataurls.txt'
f = open(fname,'w')
for url in listofurls:
    f.write(url+'\n')
f.close()

## Making the processes automated

In [60]:

# function to download files from urls in a text file
def download_files(data_file, username, password, to_folder):

    with open(data_file, "r") as data_file:
        for line in data_file:
            url = line.strip()
            filename = os.path.basename(urlparse(url).path) # method only works if URL is to a file (cf. a REST endpoint)
            if is_force_download or not os.path.isfile(to_folder+'/'+filename):
                print ("Downloading file: " + url)
                ! wget -P $to_folder --user=$username --password=$password $url
            else:
                print ("File is already downloaded: " + filename)

# setup folders
data_file = 'dataurls.txt'
to_folder = os.path.join('downloads', short_name)
is_force_download = True

# for NASA Earthdata
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')

# preview downloaded links list first
df = pd.read_csv(data_file, sep=" ",header=None)
print('found', len(df), 'links')

download_files(data_file, username, password, to_folder)

found 33 links


^C


--2023-10-03 20:43:39--  https://data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11r/2021/oco2_LtCO2_210922_B11014Ar_220728230207s.nc4
Resolving data.gesdisc.earthdata.nasa.gov (data.gesdisc.earthdata.nasa.gov)... 18.66.218.74, 18.66.218.64, 18.66.218.9, ...
Connecting to data.gesdisc.earthdata.nasa.gov (data.gesdisc.earthdata.nasa.gov)|18.66.218.74|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://urs.earthdata.nasa.gov/oauth/authorize?client_id=e2WVk8Pw6weeLUKZYOxvTQ&response_type=code&redirect_uri=https://data.gesdisc.earthdata.nasa.gov/login&state=%2Fdata%2FOCO2_DATA%2FOCO2_L2_Lite_FP.11r%2F2021%2Foco2_LtCO2_210922_B11014Ar_220728230207s.nc4&app_type=401 [following]
--2023-10-03 20:43:47--  https://urs.earthdata.nasa.gov/oauth/authorize?client_id=e2WVk8Pw6weeLUKZYOxvTQ&response_type=code&redirect_uri=https://data.gesdisc.earthdata.nasa.gov/login&state=%2Fdata%2FOCO2_DATA%2FOCO2_L2_Lite_FP.11r%2F2021%2Foco2_LtCO2_210922

--2023-10-03 20:49:08--  https://data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11r/2021/oco2_LtCO2_210923_B11014Ar_220728230243s.nc4
Resolving data.gesdisc.earthdata.nasa.gov (data.gesdisc.earthdata.nasa.gov)... 18.66.218.93, 18.66.218.64, 18.66.218.74, ...
Connecting to data.gesdisc.earthdata.nasa.gov (data.gesdisc.earthdata.nasa.gov)|18.66.218.93|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://urs.earthdata.nasa.gov/oauth/authorize?client_id=e2WVk8Pw6weeLUKZYOxvTQ&response_type=code&redirect_uri=https://data.gesdisc.earthdata.nasa.gov/login&state=%2Fdata%2FOCO2_DATA%2FOCO2_L2_Lite_FP.11r%2F2021%2Foco2_LtCO2_210923_B11014Ar_220728230243s.nc4&app_type=401 [following]
--2023-10-03 20:49:10--  https://urs.earthdata.nasa.gov/oauth/authorize?client_id=e2WVk8Pw6weeLUKZYOxvTQ&response_type=code&redirect_uri=https://data.gesdisc.earthdata.nasa.gov/login&state=%2Fdata%2FOCO2_DATA%2FOCO2_L2_Lite_FP.11r%2F2021%2Foco2_LtCO2_21092

--2023-10-03 21:08:37--  https://data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11r/2021/oco2_LtCO2_210924_B11014Ar_220728230312s.nc4
Resolving data.gesdisc.earthdata.nasa.gov (data.gesdisc.earthdata.nasa.gov)... failed: No such host is known. .
wget: unable to resolve host address 'data.gesdisc.earthdata.nasa.gov'


...

--2023-10-03 21:08:45--  https://data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11r/2021/oco2_LtCO2_211019_B11014Ar_220728224452s.nc4
Resolving data.gesdisc.earthdata.nasa.gov (data.gesdisc.earthdata.nasa.gov)... failed: No such host is known. .
wget: unable to resolve host address 'data.gesdisc.earthdata.nasa.gov'


--2023-10-03 21:08:45--  https://data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11r/2021/oco2_LtCO2_211020_B11014Ar_220728224516s.nc4
Resolving data.gesdisc.earthdata.nasa.gov (data.gesdisc.earthdata.nasa.gov)... failed: No such host is known. .
wget: unable to resolve host address 'data.gesdisc.earthdata.nasa.gov'


--2023-10-03 21:08:45--  https://data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11r/2021/oco2_LtCO2_211021_B11014Ar_220728224555s.nc4
Resolving data.gesdisc.earthdata.nasa.gov (data.gesdisc.earthdata.nasa.gov)... failed: No such host is known. .
wget: unable to resolve host address 'data.gesdisc.earthdata.nasa.gov'


--2023-10-03 21:08:46--  https://data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11r/2021/oco2_LtCO2_211022_B11014Ar_220728224619s.nc4
Resolving data.gesdisc.earthdata.nasa.gov (data.gesdisc.earthdata.nasa.gov)... failed: No such host is known. .
wget: unable to resolve host address 'data.gesdisc.earthdata.nasa.gov'


--2023-10-03 21:08:46--  https://data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11r/2021/oco2_LtCO2_211023_B11014Ar_220728224659s.nc4
Resolving data.gesdisc.earthdata.nasa.gov (data.gesdisc.earthdata.nasa.gov)... failed: No such host is known. .
wget: unable to resolve host address 'data.gesdisc.earthdata.nasa.gov'
--2023-10-03 21:08:47--  https://data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11r/2021/oco2_LtCO2_211024_B11014Ar_220728224720s.nc4
Resolving data.gesdisc.earthdata.nasa.gov (data.gesdisc.earthdata.nasa.gov)... failed: No such host is known. .
wget: unable to resolve host address 'data.gesdisc.earthdata.nasa.gov'


## Make the csv

In [ ]:
# which dataset
path_files = f'downloads/{short_name}'
year='14'
month='09'
day=6

# NZ bounding box
latMin = -48
latMax = -34
lonMin = 165
lonMax = 179

file_dict = dict_oco2_xco2
print(file_dict)

oco2_aoi = OCO2(path_files+'/*'+'_'+year+month+'*.nc4', file_dict, latMin, latMax, lonMin, lonMax)

t_unit_oco2    = "seconds since 1970-1-1 0:0:0"
t_cal = u"gregorian"

# Convert oco2 time units to a python time
oco2_aoi.time_python  = convert_time(oco2_aoi.time, t_unit_oco2, t_cal)
oco2_aoi.time_str = [datetime.datetime.fromisoformat(str(date)) for date in oco2_aoi.time_python]

df = pd.DataFrame({"Time":pd.Series(oco2_aoi.time_str).dt.date,
                   "Latitude":oco2_aoi.lat,
                   "Longitude":oco2_aoi.lon,
                   f"CO2_20{year}_(ppm)":oco2_aoi.xco2})

df_grouped = df.groupby(['Time', 'Latitude', 'Longitude'])[f"CO2_20{year}_(ppm)"].mean() # group by latitude, longitude, and date and calculate the mean of co2
df_grouped.to_csv("tret.csv")
